# Laminar Newtonian Similarity

As usual, we will begin by importing the necessary python packages

In [ ]:
from barennet import SimilarityModel
import tensorflow as tf
import numpy as np

tf.config.set_visible_devices([], 'GPU')

We will find the incomplete similarity exponents for the laminar newtonian 
mean velocity profile in wall coordinates. We know that the dimensionless quantities
involved are $(u^+, y^+, Re_\tau)$. They are related through the following equation:

$$ u^+ = y^+ - \frac{1}{2 Re_\tau} \left( y^+ \right)^2.$$

We provide a data file generated following the equation above with $10 < Re_\tau < 100$
and $0 < y^+ < Re_\tau$ (which, in bulk coordinates, translates to $0 < y < \delta$).
The MDDP construction of the three dimensionless quantities is also provided in the 
code cell below.

In [ ]:
data_path = "../Data/laminar_MVP_data.xlsx"
dimensionally_independent_parameters = ["mu", "rho", "delta"]
dimensionally_dependent_parameters = ["y", "- dp/dz"]
dimensional_qoi = "u"
non_dimensional_parameters = ["y+", "Re_tau"]
non_dimensional_qoi = "u+"
non_dimensional_params_construction = { 
    "y+": {"y": 1.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 0.5},
    "Re_tau": {"y": 0.0, "- dp/dz": 0.5, "mu": -1, "rho": 0.5, "delta": 1.5}, 
}
non_dimensional_qoi_construction = {
    "u+": {"u": 1, "y": 0.0, "- dp/dz": -0.5, "mu": 0.0, "rho": 0.5, "delta": -0.5}
}
non_similar_params = ["y+"]
similar_params = ["Re_tau"]

In [ ]:
similarity_model = SimilarityModel(
    data_path=data_path,
    dimensionally_independent_params=dimensionally_independent_parameters,
    dimensionally_dependent_params=dimensionally_dependent_parameters,
    dimensional_qoi=dimensional_qoi,
    non_dimensional_params=non_dimensional_parameters,
    non_dimensional_qoi=non_dimensional_qoi,
    non_dimensional_params_construction=non_dimensional_params_construction,
    non_dimesional_qoi_construction=non_dimensional_qoi_construction,
    non_similar_params=non_similar_params,
    similar_params=similar_params)

### Buckingham's Similarity Group

As soon as the dimensionless construction is provided and we create an instance 
of the SimilarityModel class, the Buckingham's similarity group can be calculated
by solving a linear system (chapter 5 of the thesis).
It shoud read:
\begin{align*}
    &\mu^* = A_1 \mu, \ \ \ \rho^* = A_2 \rho, \ \ \ \delta^* = A_3 \delta \\ & \\
    & y^* = A_3 y \\ & \\
    & \left( \frac{dp}{dz} \right)^* = \frac{A_1^2}{A_2 A_3^3} \left( \frac{dp}{dz} \right) \\ & \\
    &U^* = \frac{A_1}{A_2 A_3} U
\end{align*}

And, as you can see, this is exactly the output of the cell below.

In [ ]:
similarity_model.print_buckingham_similarity_group()

In [ ]:
similarity_model.find_incomplete_similarity(n_tries=5, n_epochs=1000, tol=5e-3)

In [ ]:
similarity_model.incomplete_similarity_exponents_dict

In [ ]:
similarity_model.print_renormalization_group()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
plt.rc('mathtext', fontset="cm")
import matplotlib.ticker as mtick
from matplotlib import cm # Colormaps
import matplotlib.gridspec as gridspec
import seaborn as sns

from tutorial_utils.plotting_methods import (get_laminar_data_for_plotting,
                                             plot_laminar_flow)

In [ ]:
U_plus, Y_plus, Re_tau = get_laminar_data_for_plotting()

plot_laminar_flow(U_plus, Y_plus, Re_tau)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 10))

left, bottom, width, height = [0.57, 0.17, 0.3, 0.3] 
ax2 = fig.add_axes([left, bottom, width, height])

colors = cm.tab10(np.linspace(0, 1, 10))
markers=['o','^','s','d','P','*','v','D']
line_width = 1

for i in range(len(possible_re_tau)):
    ax1.scatter(np.array(Y_plus_list[i]) * (possible_re_tau[i] ** similarity_model.incomplete_similarity_exponents_dict['y+']['Re_tau']),
                 np.array(U_plus_list[i]) * (possible_re_tau[i] ** similarity_model.incomplete_similarity_exponents_dict['u+']['Re_tau']),
                 alpha = 1, s=40, linewidth = line_width, facecolors = 'none', 
                 color = colors[i], marker = markers[i], 
                  label = r'$Re_\tau= $'+r'${}$'.format(possible_re_tau[i]))
    
    ax2.scatter(np.array(Y_plus_list[i]) * (possible_re_tau[i] ** similarity_model.incomplete_similarity_exponents_dict['y+']['Re_tau']),
                 np.array(U_plus_list[i]) * (possible_re_tau[i] ** similarity_model.incomplete_similarity_exponents_dict['u+']['Re_tau']),
                 alpha = 1, s=40, linewidth = line_width, facecolors = 'none', 
                 color = colors[i], marker = markers[i], 
                  label = r'$Re_\tau= $'+r'${}$'.format(possible_re_tau[i]))
    
font = font_manager.FontProperties(family='DejaVu Sans', weight='roman',
                                   style='normal', size = 'large',
                                   stretch = 'ultra-condensed')

ax1.legend(bbox_to_anchor=(0., 0.99), loc = 'upper left', edgecolor = 'white',
            framealpha = 0, prop = font, borderaxespad=0.)
ax1.set_xlabel(r"$y^+ / Re_\tau^{0.95}$", size = 'xx-large', fontweight = 'black')
ax1.set_ylabel(r'$u^+ / Re_\tau^{0.97}$', size = 'xx-large')
#ax1.set_ylim([5,14])
ax1.grid(False)
ax1.tick_params(axis='x', labelsize = 'large')
ax1.tick_params(axis='y', labelsize = 'large')
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['left'].set_color('black')
ax1.spines['right'].set_color('black')
ax2.spines['bottom'].set_color('black')
ax2.spines['top'].set_color('black')
ax2.spines['left'].set_color('black')
ax2.spines['right'].set_color('black')
ax2.set_xlabel(r"$\log \ \left( y^+ / Re_\tau^{0.95} \right)$", fontsize = 'large')
ax2.set_ylabel(r'$u^+ / Re_\tau^{0.97}$', fontsize = 'large')
ax2.set_xscale('log')
ax2.tick_params(axis='x', labelsize = 'medium')
ax2.tick_params(axis='y', labelsize = 'medium')
#ax2.set_xlim([0.01,1])
#ax2.set_ylim([5,14])
ax2.grid(False)

plt.savefig('tutorial_plots/laminar_flow_wall_coordinates_renormalized.pdf', 
            format='pdf', 
            dpi=1200)

plt.show()